<h1> Vegetarian Restaurant in Orange County

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Analysis](#analysis)
* [Results](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this notebook, we will try to define where is the best place in Orange County to start a Vegetarian/ Vegan Restaurant.
Vegetarianism is not a trend anymore, it has become a lifestyle for many people. Now we see many meat alternatives such as veggie burgers ( made from beans or cauliflower), big companies such as "Beyond" and "Impossible burger" made their entrance into fast-food restaurants and retail stores and now there are also lab-experiments to create an in-vitro burger.
Places like Japanese, Chinese or Indian restaurants could be considered vegetarian because many of their recipes are already that way. But if you want to try a non-meat pizza or burger wit you will probably have to look somewhere else.
With this background, we think that there is a big opportunity to place a vegetarian restaurant in a metropolis like Orange County, where a lot of people are already vegetarian and there are also people who just are eager to try new things ( like tasting a fake burger that claims to taste the same as a regular one).

## Data <a name="data"></a>

We will begin to gather the data for the postal codes and cities within Orange County to create a new data frame with the postal codes and the coordinates linked, then we will be using the Foursquare API to create a new data frame showing the most commons businesses around each area.

For example, places like 'Anaheim', where is Disneyland located, will show Themes Parks and restaurants as their Most Common business, with this info we can have a better understanding of the location of the current restaurants.
With this info gathered, we can decide if we want to invest in a highly competitive city with many restaurants or try with a more conservative approach investing in a new city with a new and fresh offer.



### Importing libraries 

In [3]:
!pip install folium==0.5.0

print('installed')


     |████████████████████████████████| 79 kB 5.6 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=edbfe6033112598fcd3750aa9f30dc26dc5a4ab466cbd6b2976a539bafadfb48
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
installed


In [4]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim

import requests

import json 
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup

import folium
print('libraries imported')

libraries imported


### Importing Data

In [5]:
!pip install uszipcode
response = requests.get("http://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?FIPS=06059")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find_all("table")
oc_df = pd.read_html(str(table))
oc_df = pd.DataFrame(oc_df[4])
oc_df.shape

from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in oc_df.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

oc_df["Latitude"] = latitude
oc_df["Longitude"] = longitude
oc_df = oc_df.dropna()

address = 'Orange County, California'

geolocator = Nominatim(user_agent="oc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Orange County are {}, {}.'.format(latitude, longitude))


     |████████████████████████████████| 378 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 613 kB/s  eta 0:00:01
  Created wheel for autopep8: filename=autopep8-1.5.4-py2.py3-none-any.whl size=45286 sha256=b12066bfa3a421f4045d63867eb2d244076401e3c9fec1930c39860b2486296c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/2c/ad/e2/f5322a230aedd0091b75ec899404e3562d8bb4e7ba0f025cbd
Successfully built autopep8
Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!
The geograpical coordinate of Orange County are 33.7500378, -117.8704931.


In [6]:
oc_df = oc_df.drop(columns = 'County')
oc_df.head()


,Zip Code,City,Latitude,Longitude
0,92694,Mission Viejo,33.55,-117.64
1,92701,Santa Ana,33.75,-117.86
3,92703,Santa Ana,33.75,-117.91
4,92704,Santa Ana,33.72,-117.91
5,92705,Cowan Heights,33.74,-117.81


### Logging into Foursquare

In [79]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:  ')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:  
CLIENT_ID: 
CLIENT_SECRET:


In [32]:
oc_data = oc_df
oc_data.loc[0, 'City']

'Mission Viejo'

In [40]:
neighborhood_latitude = oc_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = oc_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = oc_data.loc[0, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Mission Viejo are 33.55, -117.64.


### First look at OC

In [8]:

# Orange County
map_oc = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city in zip(oc_df['Latitude'], oc_df['Longitude'], oc_df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_oc)  
    
map_oc

In [9]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fedfc9b02573325d4a93f82'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Santa Ana',
  'headerFullLocation': 'Downtown Santa Ana, Santa Ana',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 40,
  'suggestedBounds': {'ne': {'lat': 33.7545378045, 'lng': -117.86509109325169},
   'sw': {'lat': 33.7455377955, 'lng': -117.87589510674832}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '512535dce4b07a4d9a74b686',
       'name': 'Crave',
       'location': {'address': '410 W 4th St',
        'lat': 33.74803063347677,
        'lng': -117.8712840447692,
        'labeledLatLngs': [{'labe

In [10]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()



/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Crave,American Restaurant,33.748031,-117.871284
1,Hopper & Burr,Café,33.748047,-117.868699
2,Irenia,Filipino Restaurant,33.748186,-117.869829
3,Mil Jugos,Latin American Restaurant,33.748896,-117.870198
4,The Good Beer Company,Brewery,33.748111,-117.870060


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
oc_venues = getNearbyVenues(names=oc_df['City'],
                                   latitudes=oc_df['Latitude'],
                                   longitudes=oc_df['Longitude']
                                  )


Mission Viejo
Santa Ana
Santa Ana
Santa Ana
Cowan Heights
Santa Ana
Santa Ana
Santa Ana
Fountain Valley
Santa Ana
Tustin
Tustin
Anaheim
Anaheim
Anaheim
Anaheim
Anaheim
Anaheim
Anaheim
Brea
Brea
Fullerton
Fullerton
Fullerton
Fullerton
Garden Grove
Garden Grove
Garden Grove


In [18]:
print(oc_venues.shape)
oc_venues.head()


(486, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mission Viejo,33.55,-117.64,Founder's Park,33.551723,-117.635941,Park
1,Mission Viejo,33.55,-117.64,Sienna Trail,33.547240,-117.637621,Trail
2,Mission Viejo,33.55,-117.64,Reyes Auto Glass Group,33.546707,-117.637886,Automotive Shop
3,Mission Viejo,33.55,-117.64,Creighton/University Park,33.552355,-117.643682,Park
4,Santa Ana,33.75,-117.86,Hidden House Coffee,33.750382,-117.863100,Coffee Shop


In [20]:
oc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Anaheim,205,205,205,205,205,205
Brea,9,9,9,9,9,9
Cowan Heights,2,2,2,2,2,2
Fountain Valley,13,13,13,13,13,13
Fullerton,95,95,95,95,95,95
Garden Grove,33,33,33,33,33,33
Mission Viejo,4,4,4,4,4,4
Santa Ana,52,52,52,52,52,52
Tustin,73,73,73,73,73,73


In [21]:
print('There are {} uniques categories.'.format(len(oc_venues['Venue Category'].unique())))

There are 157 uniques categories.


## Analyzing Each Neighborhood <a name="analysis"></a>



In [22]:

# one hot encoding
oc_onehot = pd.get_dummies(oc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
oc_onehot['Neighborhood'] = oc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [oc_onehot.columns[-1]] + list(oc_onehot.columns[:-1])
oc_onehot = oc_onehot[fixed_columns]

oc_onehot.head()

,Neighborhood,ATM,Accessories Store,Advertising Agency,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,...,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Yoga Studio
0,Mission Viejo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Mission Viejo,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Mission Viejo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mission Viejo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Santa Ana,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
print('There are {} uniques categories.'.format(len(oc_onehot['Vegetarian/Vegan Restaurant'].unique())))

NameError: name 'oc_onehot' is not defined

In [23]:
oc_onehot.shape

(486, 158)

In [24]:
oc_grouped = oc_onehot.groupby('Neighborhood').mean().reset_index()
oc_grouped

,Neighborhood,ATM,Accessories Store,Advertising Agency,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,...,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Yoga Studio
0,Anaheim,0.004878,0.009756,0.000000,0.004878,0.019512,0.014634,0.000000,0.014634,0.000000,...,0.000000,0.004878,0.004878,0.000000,0.000000,0.004878,0.004878,0.000000,0.014634,0.000000
1,Brea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Cowan Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Fountain Valley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Fullerton,0.010526,0.000000,0.000000,0.000000,0.031579,0.010526,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010526,0.010526,0.010526,0.000000,0.010526,0.010526
5,Garden Grove,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000
6,Mission Viejo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Santa Ana,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000,0.019231,...,0.000000,0.019231,0.038462,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000
8,Tustin,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.013699,0.041096,0.000000,...,0.013699,0.000000,0.000000,0.013699,0.000000,0.000000,0.013699,0.013699,0.000000,0.000000


In [25]:
num_top_venues = 5

for hood in oc_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = oc_grouped[oc_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anaheim----
                          venue  freq
0  Theme Park Ride / Attraction  0.14
1                    Theme Park  0.05
2                     Gift Shop  0.04
3                     Juice Bar  0.02
4            Mexican Restaurant  0.02


----Brea----
            venue  freq
0     Pizza Place  0.22
1  Sandwich Place  0.11
2    Liquor Store  0.11
3            Café  0.11
4    Dance Studio  0.11


----Cowan Heights----
                        venue  freq
0  Construction & Landscaping   0.5
1                        Park   0.5
2                         ATM   0.0
3                 Pizza Place   0.0
4                 Opera House   0.0


----Fountain Valley----
               venue  freq
0  Convenience Store  0.15
1       Dance Studio  0.08
2               Bank  0.08
3                Pub  0.08
4               Park  0.08


----Fullerton----
                venue  freq
0         Coffee Shop  0.06
1   Korean Restaurant  0.05
2                Park  0.05
3         Pizza Place  0.05
4  Chines

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = oc_grouped['Neighborhood']

for ind in np.arange(oc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(oc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anaheim,Theme Park Ride / Attraction,Theme Park,Gift Shop,Pizza Place,Mexican Restaurant,Ice Cream Shop,American Restaurant,Bakery,Cajun / Creole Restaurant,Juice Bar
1,Brea,Pizza Place,Café,Liquor Store,Dance Studio,Sandwich Place,Sushi Restaurant,Intersection,Grocery Store,Food Truck,Food Court
2,Cowan Heights,Construction & Landscaping,Park,Yoga Studio,Farmers Market,Football Stadium,Food Truck,Food Court,Food,Flower Shop,Fast Food Restaurant
3,Fountain Valley,Convenience Store,Auto Garage,Chinese Restaurant,Pub,Sushi Restaurant,Park,Thai Restaurant,Bank,Gluten-free Restaurant,Gym / Fitness Center
4,Fullerton,Coffee Shop,Park,Pizza Place,Korean Restaurant,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Sushi Restaurant,Sandwich Place,Grocery Store


In [76]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Anaheim,Theme Park Ride / Attraction,Theme Park,Gift Shop,Pizza Place,Mexican Restaurant,Ice Cream Shop,American Restaurant,Bakery,Cajun / Creole Restaurant,Juice Bar
1,4,Brea,Pizza Place,Café,Liquor Store,Dance Studio,Sandwich Place,Sushi Restaurant,Intersection,Grocery Store,Food Truck,Food Court
2,2,Cowan Heights,Construction & Landscaping,Park,Yoga Studio,Farmers Market,Football Stadium,Food Truck,Food Court,Food,Flower Shop,Fast Food Restaurant
3,0,Fountain Valley,Convenience Store,Auto Garage,Chinese Restaurant,Pub,Sushi Restaurant,Park,Thai Restaurant,Bank,Gluten-free Restaurant,Gym / Fitness Center
4,3,Fullerton,Coffee Shop,Park,Pizza Place,Korean Restaurant,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Sushi Restaurant,Sandwich Place,Grocery Store
5,3,Garden Grove,Mexican Restaurant,Fast Food Restaurant,Pizza Place,Pharmacy,Hotel,Rental Car Location,Sandwich Place,Coffee Shop,Fried Chicken Joint,Pool
6,1,Mission Viejo,Park,Automotive Shop,Trail,Yoga Studio,Football Stadium,Food Truck,Food Court,Food,Flower Shop,Fast Food Restaurant
7,3,Santa Ana,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
8,3,Tustin,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Sushi Restaurant,Asian Restaurant,Italian Restaurant,Discount Store,Indian Restaurant,Ice Cream Shop,Rental Car Location


 ### Note:  
 At first glance we can see that only Tustin has Vegetarian places (asian, indian, japanese restaurants)<br>
 It seems that we have an opportunity to start a business in the other 8 Neighborhoods.

### Clustering Neighborhoods/Cities¶


In [70]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

oc_grouped_clustering = oc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(oc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 4, 2, 0, 3, 3, 1, 3, 3], dtype=int32)

In [71]:
oc_merged = oc_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
oc_merged = oc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='City')

oc_merged.head() # check the last columns!

,Zip Code,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,92694,Mission Viejo,33.55,-117.64,1,Park,Automotive Shop,Trail,Yoga Studio,Football Stadium,Food Truck,Food Court,Food,Flower Shop,Fast Food Restaurant
1,92701,Santa Ana,33.75,-117.86,3,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
3,92703,Santa Ana,33.75,-117.91,3,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
4,92704,Santa Ana,33.72,-117.91,3,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
5,92705,Cowan Heights,33.74,-117.81,2,Construction & Landscaping,Park,Yoga Studio,Farmers Market,Football Stadium,Food Truck,Food Court,Food,Flower Shop,Fast Food Restaurant


In [72]:
oc_merged.dropna(subset = ['City','Cluster Labels', 'Latitude','Longitude'], inplace=True)


## Results <a name="results"></a>


In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(oc_merged['Latitude'], oc_merged['Longitude'], oc_merged['City'], oc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
oc_merged.loc[oc_merged['Cluster Labels'] == 0, oc_merged.columns[[1] + list(range(5, oc_merged.shape[1]))]]


,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Fountain Valley,Convenience Store,Auto Garage,Chinese Restaurant,Pub,Sushi Restaurant,Park,Thai Restaurant,Bank,Gluten-free Restaurant,Gym / Fitness Center


In [58]:
oc_merged.loc[oc_merged['Cluster Labels'] == 1, oc_merged.columns[[1] + list(range(5, oc_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mission Viejo,Park,Automotive Shop,Trail,Yoga Studio,Football Stadium,Food Truck,Food Court,Food,Flower Shop,Fast Food Restaurant


In [59]:
oc_merged.loc[oc_merged['Cluster Labels'] == 2, oc_merged.columns[[1] + list(range(5, oc_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Cowan Heights,Construction & Landscaping,Park,Yoga Studio,Farmers Market,Football Stadium,Food Truck,Food Court,Food,Flower Shop,Fast Food Restaurant


In [60]:
oc_merged.loc[oc_merged['Cluster Labels'] == 3, oc_merged.columns[[1] + list(range(5, oc_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Santa Ana,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
3,Santa Ana,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
4,Santa Ana,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
6,Santa Ana,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
7,Santa Ana,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
8,Santa Ana,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
10,Santa Ana,Park,Convenience Store,Mexican Restaurant,Train Station,Construction & Landscaping,Marijuana Dispensary,Food,Flower Shop,Fast Food Restaurant,Chinese Restaurant
16,Tustin,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Sushi Restaurant,Asian Restaurant,Italian Restaurant,Discount Store,Indian Restaurant,Ice Cream Shop,Rental Car Location
18,Tustin,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Sushi Restaurant,Asian Restaurant,Italian Restaurant,Discount Store,Indian Restaurant,Ice Cream Shop,Rental Car Location
20,Anaheim,Theme Park Ride / Attraction,Theme Park,Gift Shop,Pizza Place,Mexican Restaurant,Ice Cream Shop,American Restaurant,Bakery,Cajun / Creole Restaurant,Juice Bar


In [61]:
oc_merged.loc[oc_merged['Cluster Labels'] == 4, oc_merged.columns[[1] + list(range(5, oc_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Brea,Pizza Place,Café,Liquor Store,Dance Studio,Sandwich Place,Sushi Restaurant,Intersection,Grocery Store,Food Truck,Food Court
37,Brea,Pizza Place,Café,Liquor Store,Dance Studio,Sandwich Place,Sushi Restaurant,Intersection,Grocery Store,Food Truck,Food Court


### Conclusion: <a name='conclusion'></a>


- Places like 'Fountain Valey','Fullerton' and 'Tustin' already have vegetarian options.

- 'Anaheim' could be a good place because of the level of attendance to the theme parks and you can find people looking for a broad variety of food options. This option could be expensive if you want to be near the attractions.

- 'Santa Ana' seems like a good place to start a vegetarian restaurant business since there is only a convenience store and Mexican restaurant as their most common business, we may have an opportunity to offer a different menu to the people there.